In [1]:
import mlflow
from mlflow.tracking import MlflowClient

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

client = MlflowClient()

# Use the search_experiments function to get a list of available experiments
experiments = client.search_experiments()
print(len(experiments))

# Display the list of available experiments
for experiment in experiments:
    print(f"Experiment Name: {experiment.name}, Experiment ID: {experiment.experiment_id}")

tracking URI: 'file:///Users/mohammedzaidsyed/Desktop/Spam/spamclassification/mlruns'
3
Experiment Name: SVM, Experiment ID: 407553083182888926
Experiment Name: Random Forest, Experiment ID: 266853881450216497
Experiment Name: Naive Bayes, Experiment ID: 831114447560693852


In [2]:
import pandas as pd
def load_data(path):
    return pd.read_csv(path)

df = load_data('data/spam_ham_dataset.csv')
df.head()
# df.shape

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [3]:
df.columns

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt

# Drop any rows with missing values
df.dropna(inplace=True)

# Encoding categorical variables
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Convert all non-numeric columns to strings
df = df.applymap(str)

# Vectorize text data if needed
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(df['text'])

# Identify X and y variables
X = pd.concat([df.drop(columns=['label_num', 'text']), pd.DataFrame(X_text.toarray())], axis=1)
y = df['label_num'].astype(float)


In [5]:
df.columns

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')

In [6]:
# df.describe()
df.nunique()
# df.isna().sum()

Unnamed: 0    5171
label            2
text          4993
label_num        2
dtype: int64

In [10]:
from typing import List
import pickle
import os
import mlflow
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

def encode_cols(df: pd.DataFrame, categorical_cols: List[str] = None) -> pd.DataFrame:
    if categorical_cols is None:
        categorical_cols = ["label", "text", "label_num"]
    
    # Iterate over each column name in the list
    for col in categorical_cols:
        df[col] = df[col].astype(str).str.lower()
    
    return df


def extract_x_y(
    df: pd.DataFrame,
    categorical_cols: List[str] = None,
    numerical_cols: List[str] = None,
    dv: DictVectorizer = None,
    with_target: bool = True,
) -> dict:
    if categorical_cols is None:
        categorical_cols = ["label", "text", "label_num"]
    
    dicts = df[[*categorical_cols]].to_dict(orient="records")

    y = None
    if with_target:
        if dv is None:
            dv = DictVectorizer()
            dv.fit(dicts)
        y = df["label"].values

    x = dv.transform(dicts)
    return x, y, dv

def save_pickle(path: str, file):
    with open(path, "wb") as f:
        pickle.dump(file, f)

# save the preprocessor into saved_pkl folder
import pickle
def save_picked(path: str, file):
    with open(path, "wb") as f:
        pickle.dump(file, f)